#### Import packages and Library

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
#library
import library_final
from library_final.pokedex import DataLoader as ld
from library_final.pokedex import DataPreprocessor as nc
from library_final.pokedex import FeatureEng as en
from library_final.pokedex import ModelSelectorFoo as ml1
from library_final.pokedex import ModelSelector_copy as ml2


### Libraries for Database SQL
import pandas.io.sql as sqlio
import psycopg2
import json
from sqlalchemy import create_engine

#### Loading Datasets

1. Load datasets
  * Pokemon
  * Combats
  * Pokemon_id_each_team
  * Team_combat

In [2]:
## Load from Database
loader = ld.loader_database()
combats, pokemon, pokemon_id_each_team, team_combat, ProcessedData = loader.load_datasets()
## Load from repository
#loader = ld.loader_jupyter()
#combats, pokemon, pokemon_id_each_team, team_combat, Processed = loader.load_datasets()

#### Preprocessing & Feature Transformation

#### Numeric
1. Calculating Combat metrics
  * Victory_Rate
  * Total_Battles
  * Victory_Counts

2. Calculating pokemon skill levels
  * offensive_calculator
  * defensive_calculator
  * speed_power_ratio_calculator

#### Categorical
2. One hot encoding
  * Type 1
  * Type 2

3. Label Encoding
  * Legendary

In [3]:
## Creating new columns
combat_cols = nc.PokemonBattleProcessor(pokemon,combats)
combat_cols.process_battle_data()
pokemon_dt = combat_cols.get_processed_data()

In [4]:
pokemon_dt

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Victory_Rate,Total_Battles,Victory_Counts
0,1.0,Bulbasaur,Grass,Poison,45.0,49.0,49.0,65.0,65.0,45.0,1.0,False,0.278195,133.0,37.0
1,2.0,Ivysaur,Grass,Poison,60.0,62.0,63.0,80.0,80.0,60.0,1.0,False,0.380165,121.0,46.0
2,3.0,Venusaur,Grass,Poison,80.0,82.0,83.0,100.0,100.0,80.0,1.0,False,0.674242,132.0,89.0
3,4.0,Mega Venusaur,Grass,Poison,80.0,100.0,123.0,122.0,120.0,80.0,1.0,False,0.560000,125.0,70.0
4,5.0,Charmander,Fire,None,39.0,52.0,43.0,60.0,50.0,65.0,1.0,False,0.491071,112.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796.0,Diancie,Rock,Fairy,50.0,100.0,150.0,100.0,150.0,50.0,6.0,True,0.371429,105.0,39.0
796,797.0,Mega Diancie,Rock,Fairy,50.0,160.0,110.0,160.0,110.0,110.0,6.0,True,0.885496,131.0,116.0
797,798.0,Hoopa Confined,Psychic,Ghost,80.0,110.0,60.0,150.0,130.0,70.0,6.0,True,0.504202,119.0,60.0
798,799.0,Hoopa Unbound,Psychic,Dark,80.0,160.0,60.0,170.0,130.0,80.0,6.0,True,0.618056,144.0,89.0


In [5]:
#Setting Pokemon Calculator Class
offensive_calculator = nc.OffensivePowerCalculator(pokemon_dt)
defensive_calculator = nc.DefensivePowerCalculator(pokemon_dt)
speed_power_ratio_calculator = nc.SpeedToPowerRatioCalculator(pokemon_dt)

#Calculating Pokemon's OffensivePower
offensive_calculator.calculate()
#Calculating Pokemon's DefensivePower
defensive_calculator.calculate()
#Calculating Pokemon's SpeedToPower
speed_power_ratio_calculator.calculate()

In [6]:
pokemon_dt.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary', 'Victory_Rate',
       'Total_Battles', 'Victory_Counts', 'Offensive_Power', 'Defensive_Power',
       'Speed_to_Power_Ratio'],
      dtype='object')

In [7]:
#Checkpoint - Creating a Copy before starting one hot encoding
type_encoder = en.PokemonTypeEncoder(pokemon_dt)
#Create Dummies for Type 1 category variable
type_encoder.one_hot_encode_type1()
# Create new column only if the type doesn't exist
type_encoder.encode_type2()
# Return a new DataFrame with the update pokemon dataset + dummies
df = type_encoder.get_updated_dataframe()

In [8]:
#Label Encode Legendary Column
legend_encoder = en.Pokemon_Dummy_Legendary_Encoder(df)
legend_encoder.encode_legendary()

In [9]:
df.iloc[:,9:].head(5)

,Speed,Generation,Legendary,Victory_Rate,Total_Battles,Victory_Counts,Offensive_Power,Defensive_Power,Speed_to_Power_Ratio,Bug,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,45.0,1.0,0,0.278195,133.0,37.0,0.248473,0.189074,0.425051,0,...,0,1,0,0,0,1,0,0,0,0
1,60.0,1.0,0,0.380165,121.0,46.0,0.329279,0.252015,0.455788,0,...,0,1,0,0,0,1,0,0,0,0
2,80.0,1.0,0,0.674242,132.0,89.0,0.440622,0.337417,0.472856,0,...,0,1,0,0,0,1,0,0,0,0
3,80.0,1.0,0,0.560000,125.0,70.0,0.494838,0.405671,0.381560,0,...,0,1,0,0,0,1,0,0,0,0
4,65.0,1.0,0,0.491071,112.0,55.0,0.302877,0.152740,0.650838,0,...,0,0,0,0,0,0,0,0,0,0


### Model Selection

1. Split Train and Test
2. Select Model
3. Train Model and Make Predictions
4. Model Evaluation
5. K_fold_cross_validation

In [10]:
# Dividing Dataset into Train and Test
Data_split = ml1.TrainTestDivider(df)
X_train, X_test, y_train, y_test = Data_split.train_test()

#### Linear Regression

In [12]:
linear_reg_model = ml1.LinearRegressionAnalyzer()
linear_reg_model.set_data(X_train, y_train, X_test, y_test)
linear_reg_model.simple_result()

Model trained successfully.
Mean Squared Error: 0.009378679186597561 and R-squared: 0.8553790975705335


#### Hyperparameter Tunning (Lasso Optimizer)

In [13]:
# Step 1: Create an instance of LassoModel
lasso_model = ml2.LassoModel(alpha=0.0001)  # You can set the alpha value

# Step 2: Run the Lasso model without cross-validation
lasso_model.run_model(X_train, y_train, X_test, y_test)

# Step 3: Run the Lasso model with cross-validation
lasso_model.run_model_cv(df_X, df_y, n_splits=5)  # Set the number of splits as needed

Mean Squared Error: 0.009384491757651119 and R-squared: 0.8552894666902703
   
Cross-Validation r_squared Scores: [0.80569763 0.91932673 0.79120739 0.83051764 0.87571801]
Mean Cross-Validation r_squared: 0.8444934786181664

Cross-Validation mse Scores: [0.01351886 0.00571058 0.01432478 0.01106223 0.00822008]
Mean Cross-Validation mse: 0.010567306176495286



#### Hyperparameter Tunning (Ridge Optimizer)

In [16]:
# Create Lasso optimizer class
model2 = ml.RidgeOptimizer(X_train, y_train, X_test, y_test)

# Optimize alpha without CV
alphas_ridge = np.linspace(0.0001, 1, 100) # Define alpha range to run optimization
model2.optimize_alpha(alphas_ridge)

Optimal alpha for Ridge without CV: 0.0001 and its average MSE: 0.009378679376585767


In [17]:
# Optimize alpha with CV
model2.optimize_alpha_cv(alphas_ridge)

Optimal alpha for Ridge with CV: 1.0 and its MSE: 0.009374400774136728
